In [3]:
import os
from keras.models import Model
#from keras.optimizers import Adam
#from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np
import tensorflow as tf

In [5]:
BATCH_SIZE = 32

train_generator = ImageDataGenerator(rotation_range=.15, 
                                     #brightness_range=[0.1, 0.7],
                                     width_shift_range=0.1, 
                                     height_shift_range=0.1,
                                     horizontal_flip=True, 
                                     #vertical_flip=True,
                                     validation_split=0.20,
                                     preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

In [6]:
train_data_dir = 'C:/Users/colorlab/Downloads/Emotion_Classes-20211030T160812Z-001/Emotion_Classes'
class_subset = sorted(os.listdir(train_data_dir))[:10]
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(48, 48),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               #color_mode = 'grayscale',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(48, 48),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               #color_mode = 'grayscale',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)


Found 10480 images belonging to 10 classes.
Found 2620 images belonging to 10 classes.


In [7]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    # MobileNetV2(include_top=False,
     #                weights='imagenet', 
      #               input_shape=)
    
    conv_base = tf.keras.applications.densenet.DenseNet121(
    input_shape=input_shape, include_top=False, weights='imagenet'
)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(1524, activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    top_model = Dense(1524, activation='relu')(top_model)
    top_model = Dropout(0.4)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [8]:
#from adabound import AdaBound
import tensorflow as tf
input_shape = (48, 48, 3)

n_classes=10
optim_1 = tf.keras.optimizers.Adam(
    learning_rate=0.001
)
n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 60

# First we'll train the model without Fine-tuning
mobilenetv2_model = create_model(input_shape, n_classes, optim_1, fine_tune=6)


In [9]:

mobilenetv2_model = mobilenetv2_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                           # callbacks=[tl_checkpoint_1, early_stop, plot_losses],
                            verbose=1)

Epoch 1/60
327/327 [==============================] - 33s 90ms/step - loss: 1.9720 - accuracy: 0.3209 - val_loss: 1.6646 - val_accuracy: 0.4371
Epoch 2/60
327/327 [==============================] - 29s 88ms/step - loss: 1.7307 - accuracy: 0.3961 - val_loss: 1.5852 - val_accuracy: 0.4452
Epoch 3/60
327/327 [==============================] - 29s 89ms/step - loss: 1.6451 - accuracy: 0.4245 - val_loss: 1.4899 - val_accuracy: 0.4973
Epoch 4/60
327/327 [==============================] - 29s 90ms/step - loss: 1.5795 - accuracy: 0.4521 - val_loss: 1.5440 - val_accuracy: 0.4637
Epoch 5/60
327/327 [==============================] - 30s 91ms/step - loss: 1.5616 - accuracy: 0.4591 - val_loss: 1.4649 - val_accuracy: 0.4992
Epoch 6/60
327/327 [==============================] - 30s 90ms/step - loss: 1.5014 - accuracy: 0.4804 - val_loss: 1.4510 - val_accuracy: 0.4969
Epoch 7/60
327/327 [==============================] - 30s 91ms/step - loss: 1.4817 - accuracy: 0.4857 - val_loss: 1.4006 - val_accuracy:

###### 